In [11]:
import os
os.path.abspath("")

'C:\\Users\\rajat\\Capstone'

In [12]:
#!ls -l 'deploy.prototxt'
#!chmod 644 'deploy.prototxt

In [13]:
import cv2
import dlib
import numpy as np
import playsound


In [14]:
# Load the yawn detector
yawn_detector = cv2.CascadeClassifier("haarcascade_mcs_mouth.xml")

In [15]:

# Define the facial landmark points for the left and right eyes
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

In [16]:
def calculate_EAR(eye_points):
    """
    Calculate the Eye Aspect Ratio (EAR) given the eye landmark points.
    """
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [17]:
# Load the dlib face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('new_model.dat')


In [18]:
alarm = ("alarm.wav")

In [19]:
# Load the video capture device
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect the face in the grayscale frame
    faces = detector(gray)
    
    # Loop through each face
    for face in faces:
        # Get the facial landmarks for the face
        landmarks = predictor(gray, face)
        
        # Get the landmark points for the left and right eyes
        left_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in LEFT_EYE_POINTS])
        right_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in RIGHT_EYE_POINTS])
        
        # Calculate the Eye Aspect Ratio (EAR) for each eye
        left_ear = calculate_EAR(left_eye_points)
        right_ear = calculate_EAR(right_eye_points)
        
        # Calculate the average EAR for both eyes
        avg_ear = (left_ear + right_ear) / 2.0
        
        # Determine if the eyes are closed or open based on the EAR threshold
        if avg_ear < 0.2:
            Closed = cv2.putText(frame, "Eyes Closed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            print (Closed)
            playsound(alarm)


        else:
            cv2.putText(frame, "Eyes Open", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Draw the landmarks and eye regions on the frame
        for i in LEFT_EYE_POINTS:
            cv2.circle(frame, (landmarks.part(i).x, landmarks.part(i).y), 2, (0, 255, 0), -1)
        for i in RIGHT_EYE_POINTS:
            cv2.circle(frame, (landmarks.part(i).x, landmarks.part(i).y), 2, (0, 255, 0), -1)
        cv2.polylines(frame, [left_eye_points], True, (0, 255, 0), 2)
        cv2.polylines(frame, [right_eye_points], True, (0, 255, 0), 2)
        
        
        # Get the facial landmarks for the face to detect Yawning 
        landmarks = predictor(gray, face)

        # Get the mouth region from the facial landmarks
        mouth_region = landmarks.parts()[60:68]

        # Convert the mouth region to a numpy array
        mouth_points = np.array([[p.x, p.y] for p in mouth_region])

        # Calculate the width and height of the mouth region
        w = np.abs(mouth_points[0][0] - mouth_points[6][0])
        h = np.abs(mouth_points[2][1] - mouth_points[7][1])

        if len(yawn_detector.detectMultiScale(gray, 1.1, 5)) > 0 and h/w > 0.2:
            yawning = cv2.putText(frame, "Yawning", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print (yawning)
            playsound(alarm)
        else:
            cv2.putText(frame, "No Yawning", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    

    # Show the frame
    cv2.imshow("Drowsiness Detector", frame)
    
    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

        
# cleanup
cv2.destroyAllWindows()


[[[112 117 115]
  [115 118 113]
  [115 117 110]
  ...
  [114 101 148]
  [120 110 152]
  [107  98 139]]

 [[114 118 113]
  [114 117 111]
  [115 117 110]
  ...
  [109  94 144]
  [107  94 141]
  [102  89 136]]

 [[116 118 113]
  [114 117 111]
  [114 118 111]
  ...
  [103  91 136]
  [106  96 140]
  [102  92 136]]

 ...

 [[ 33  34  40]
  [ 29  30  36]
  [ 29  31  35]
  ...
  [200 184 163]
  [202 184 163]
  [199 181 160]]

 [[ 27  25  34]
  [ 27  23  33]
  [ 27  22  32]
  ...
  [205 187 165]
  [203 187 166]
  [199 184 163]]

 [[ 27  27  36]
  [ 27  26  35]
  [ 26  24  33]
  ...
  [213 193 170]
  [213 193 170]
  [201 182 158]]]
[[[115 116 112]
  [114 116 112]
  [112 115 111]
  ...
  [112  97 151]
  [121 105 154]
  [113  97 145]]

 [[115 116 112]
  [115 116 112]
  [115 116 112]
  ...
  [106  90 144]
  [106  89 141]
  [106  89 141]]

 [[115 116 112]
  [116 117 113]
  [115 116 112]
  ...
  [107  94 136]
  [106  94 134]
  [105  93 133]]

 ...

 [[ 42  38  43]
  [ 37  34  38]
  [ 30  27  30]
  ..

In [10]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print((i,dev['name'],dev['maxInputChannels']))
p.terminate()

(0, 'Microsoft Sound Mapper - Input', 2)
(1, 'Microphone Array (IntelÂ® Smart ', 4)
(2, 'Microsoft Sound Mapper - Output', 0)
(3, 'Speakers (Realtek(R) Audio)', 0)
(4, 'Primary Sound Capture Driver', 2)
(5, 'Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)', 4)
(6, 'Primary Sound Driver', 0)
(7, 'Speakers (Realtek(R) Audio)', 0)
(8, 'Speakers (Realtek(R) Audio)', 0)
(9, 'Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)', 2)
(10, 'Speakers 1 (Realtek HD Audio output with SST)', 0)
(11, 'Speakers 2 (Realtek HD Audio output with SST)', 0)
(12, 'PC Speaker (Realtek HD Audio output with SST)', 2)
(13, 'Stereo Mix (Realtek HD Audio Stereo input)', 2)
(14, 'Headphones 1 (Realtek HD Audio 2nd output with SST)', 0)
(15, 'Headphones 2 (Realtek HD Audio 2nd output with SST)', 0)
(16, 'PC Speaker (Realtek HD Audio 2nd output with SST)', 2)
(17, 'Microphone (Realtek HD Audio Mic input)', 2)
(18, 'Microphone Array 1 ()', 2)
(19, 'Microphone Array 2 (